<details>
Modelo a vencer:

```
params_model_1 = {
  "max_bin": 100,
  "num_leaves": 75,
  "learning_rate": 0.006817310481237339,
  "min_data_in_leaf": 50,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 1700,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True
}

_, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103])
```

Primera prueba: \[ensamble de modelo con params_model_1, features filtradas y febrero-marzo\] + \[modelo de enero\]

Params por default:
```
  {"max_bin": 100,
  "num_leaves": 75,
  "learning_rate": 0.006817310481237339,
  "min_data_in_leaf": 50,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 1700,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True,
  "force_row_wise": True}
```
</details>

# Carga de cosas

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import re
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import gc
#import optuna
import itertools
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed, parallel_backend
from time import time
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from multiprocessing import Pool, cpu_count
import itertools
#from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

In [6]:
base_path = '/content/drive/MyDrive/EyF/'
dataset_path = base_path + 'datos/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'fe_std.parquet'
dataset_file = 'fe_0210.parquet'
dataset_file = 'fe_muerte.parquet'
dataset_file = 'fe_muerte_v2.parquet'
dataset_file = 'fe_muerte_v3.parquet'
dataset_file = 'fe_muerte_v5.parquet'
dataset_file = 'fe_muerte_v6.parquet'
EXPERIMENTS_DIR = '/content/drive/MyDrive/EyF/experimentos/gbm/'

## Mes train

In [7]:
mes_train = 202103
mes_test = 202104

# agregue sus semillas
semillas = [104801, 104803, 114809, 134807, 148013]

# X_train = data[data['foto_mes'] == mes_train]
# y_train = X_train['clase_ternaria']
# X_train = X_train.drop(columns = ['clase_ternaria'])

In [8]:
# train_data = data[data['foto_mes'] == mes_train]
# test_data = data[data['foto_mes'] == mes_test]

# X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
# y_train_binaria1 = train_data['clase_binaria1']
# y_train_binaria2 = train_data['clase_binaria2']
# w_train = train_data['clase_peso']

# X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
# y_test_binaria1 = test_data['clase_binaria1']
# y_test_class = test_data['clase_ternaria']
# w_test = test_data['clase_peso']

## Subsets

In [9]:
seleccion = ['std_ctrx_quarter', 'ctrx_quarter', 'mcaja_ahorro', 'cpayroll_trx', 'mpasivos_margen', 'mcuentas_saldo', 'std_mpayroll', 'std_mprestamos_personales', 'mprestamos_personales', 'mpayroll', 'std_cpayroll_trx', 'std_mtarjeta_visa_consumo', 'Visa_status', 'std_mpasivos_margen', 'mcuenta_corriente', 'Visa_msaldopesos', 'Visa_msaldototal', 'delta_1m_ccomisiones_mantenimiento', 'ratio_mprestamos_personales_vs_prev', 'mtarjeta_visa_consumo', 'Master_status', 'cdescubierto_preacordado', 'lag_1_mrentabilidad_annual', 'std_Visa_msaldopesos', 'lag_1_ctrx_quarter', 'Visa_fechaalta', 'std_mcuenta_corriente', 'mactivos_margen', 'Visa_mpagominimo', 'Master_fechaalta', 'cliente_edad', 'std_mrentabilidad_annual', 'mrentabilidad_annual', 'ratio_cproductos_vs_prev', 'std_mcuentas_saldo', 'cproductos', 'delta_2m_ccomisiones_mantenimiento', 'delta_2m_mcomisiones_mantenimiento', 'chomebanking_transacciones', 'std_mactivos_margen', 'delta_1m_mrentabilidad_annual', 'mtarjeta_master_consumo', 'delta_1m_mcomisiones_mantenimiento', 'ccaja_seguridad', 'lag_1_mcuentas_saldo', 'tmobile_app', 'ratio_mcuentas_saldo_vs_prev', 'ccallcenter_transacciones', 'std_cprestamos_personales', 'internet', 'delta_1m_mcuentas_saldo', 'lag_2_mcuenta_corriente', 'ratio_ctrx_quarter_vs_prev', 'Master_mpagominimo', 'lag_1_mcaja_ahorro', 'lag_1_mcuenta_corriente', 'std_Master_status', 'cprestamos_personales', 'std_ctarjeta_visa_transacciones', 'delta_1m_mcaja_ahorro_dolares', 'Master_Fvencimiento', 'ctarjeta_visa_transacciones', 'Master_mfinanciacion_limite']

In [10]:
features_base = ['foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales', 'ccaja_seguridad', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'mcuenta_debitos_automaticos', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'cpagodeservicios', 'mpagodeservicios', 'cpagomiscuentas', 'mpagomiscuentas', 'ccajeros_propios_descuentos', 'mcajeros_propios_descuentos', 'ctarjeta_visa_descuentos', 'mtarjeta_visa_descuentos', 'ctarjeta_master_descuentos', 'mtarjeta_master_descuentos', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento', 'ccomisiones_otras', 'mcomisiones_otras', 'cforex', 'cforex_buy', 'mforex_buy', 'cforex_sell', 'mforex_sell', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'ctransferencias_emitidas', 'mtransferencias_emitidas', 'cextraccion_autoservicio', 'mextraccion_autoservicio', 'ccheques_depositados', 'mcheques_depositados', 'ccheques_emitidos', 'mcheques_emitidos', 'ccheques_depositados_rechazados', 'mcheques_depositados_rechazados', 'ccheques_emitidos_rechazados', 'mcheques_emitidos_rechazados', 'tcallcenter', 'ccallcenter_transacciones', 'thomebanking', 'chomebanking_transacciones', 'ccajas_transacciones', 'ccajas_consultas', 'ccajas_depositos', 'ccajas_extracciones', 'ccajas_otras', 'catm_trx', 'matm', 'catm_trx_other', 'matm_other', 'ctrx_quarter', 'tmobile_app', 'cmobile_app_trx', 'Master_delinquency', 'Master_status', 'Master_mfinanciacion_limite', 'Master_Fvencimiento', 'Master_Finiciomora', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_fultimo_cierre', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_fechaalta', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo', 'Visa_delinquency', 'Visa_status', 'Visa_mfinanciacion_limite', 'Visa_Fvencimiento', 'Visa_Finiciomora', 'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 'Visa_madelantodolares', 'Visa_fultimo_cierre', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_fechaalta', 'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo']

In [11]:
std = ['std_ctrx_quarter', 'std_mcaja_ahorro', 'std_mpasivos_margen', 'std_mcuentas_saldo', 'std_cpayroll_trx', 'std_mprestamos_personales', 'std_mtarjeta_visa_consumo', 'std_mcuenta_corriente', 'std_cprestamos_personales', 'std_ctarjeta_visa_transacciones', 'std_mrentabilidad_annual', 'std_Visa_msaldopesos', 'std_mpayroll', 'std_Master_status', 'std_mactivos_margen']

In [12]:
columns_old = ['foto_mes', 'numero_de_cliente', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales', 'ccaja_seguridad', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'mcuenta_debitos_automaticos', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'cpagodeservicios', 'mpagodeservicios', 'cpagomiscuentas', 'mpagomiscuentas', 'ccajeros_propios_descuentos', 'mcajeros_propios_descuentos', 'ctarjeta_visa_descuentos', 'mtarjeta_visa_descuentos', 'ctarjeta_master_descuentos', 'mtarjeta_master_descuentos', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento', 'ccomisiones_otras', 'mcomisiones_otras', 'cforex', 'cforex_buy', 'mforex_buy', 'cforex_sell', 'mforex_sell', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 'ctransferencias_emitidas', 'mtransferencias_emitidas', 'cextraccion_autoservicio', 'mextraccion_autoservicio', 'ccheques_depositados', 'mcheques_depositados', 'ccheques_emitidos', 'mcheques_emitidos', 'ccheques_depositados_rechazados', 'mcheques_depositados_rechazados', 'ccheques_emitidos_rechazados', 'mcheques_emitidos_rechazados', 'tcallcenter', 'ccallcenter_transacciones', 'thomebanking', 'chomebanking_transacciones', 'ccajas_transacciones', 'ccajas_consultas', 'ccajas_depositos', 'ccajas_extracciones', 'ccajas_otras', 'catm_trx', 'matm', 'catm_trx_other', 'matm_other', 'ctrx_quarter', 'tmobile_app', 'cmobile_app_trx', 'Master_delinquency', 'Master_status', 'Master_mfinanciacion_limite', 'Master_Fvencimiento', 'Master_Finiciomora', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_fultimo_cierre', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_fechaalta', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo', 'Visa_delinquency', 'Visa_status', 'Visa_mfinanciacion_limite', 'Visa_Fvencimiento', 'Visa_Finiciomora', 'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 'Visa_madelantodolares', 'Visa_fultimo_cierre', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_fechaalta', 'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo', 'lag_1_mpasivos_margen', 'lag_2_mpasivos_margen', 'lag_3_mpasivos_margen', 'lag_1_cproductos', 'lag_2_cproductos', 'lag_3_cproductos', 'lag_1_mcuenta_corriente', 'lag_2_mcuenta_corriente', 'lag_3_mcuenta_corriente', 'lag_1_cdescubierto_preacordado', 'lag_2_cdescubierto_preacordado', 'lag_3_cdescubierto_preacordado', 'lag_1_mcuentas_saldo', 'lag_2_mcuentas_saldo', 'lag_3_mcuentas_saldo', 'lag_1_ctarjeta_visa_transacciones', 'lag_2_ctarjeta_visa_transacciones', 'lag_3_ctarjeta_visa_transacciones', 'lag_1_mtarjeta_visa_consumo', 'lag_2_mtarjeta_visa_consumo', 'lag_3_mtarjeta_visa_consumo', 'lag_1_mprestamos_personales', 'lag_2_mprestamos_personales', 'lag_3_mprestamos_personales', 'lag_1_cpayroll_trx', 'lag_2_cpayroll_trx', 'lag_3_cpayroll_trx', 'lag_1_ccomisiones_mantenimiento', 'lag_2_ccomisiones_mantenimiento', 'lag_3_ccomisiones_mantenimiento', 'lag_1_ccallcenter_transacciones', 'lag_2_ccallcenter_transacciones', 'lag_3_ccallcenter_transacciones', 'lag_1_ccajas_consultas', 'lag_2_ccajas_consultas', 'lag_3_ccajas_consultas', 'lag_1_ctrx_quarter', 'lag_2_ctrx_quarter', 'lag_3_ctrx_quarter', 'lag_1_Master_status', 'lag_2_Master_status', 'lag_3_Master_status', 'lag_1_Master_mfinanciacion_limite', 'lag_2_Master_mfinanciacion_limite', 'lag_3_Master_mfinanciacion_limite', 'lag_1_Visa_status', 'lag_2_Visa_status', 'lag_3_Visa_status', 'lag_1_Visa_msaldototal', 'lag_2_Visa_msaldototal', 'lag_3_Visa_msaldototal', 'delta_1m_mpasivos_margen', 'delta_1m_cproductos', 'delta_1m_mcuenta_corriente', 'delta_1m_cdescubierto_preacordado', 'delta_1m_mcuentas_saldo', 'delta_1m_ctarjeta_visa_transacciones', 'delta_1m_mtarjeta_visa_consumo', 'delta_1m_mprestamos_personales', 'delta_1m_cpayroll_trx', 'delta_1m_ccomisiones_mantenimiento', 'delta_1m_ccallcenter_transacciones', 'delta_1m_ccajas_consultas', 'delta_1m_ctrx_quarter', 'delta_1m_Master_status', 'delta_1m_Master_mfinanciacion_limite', 'delta_1m_Visa_status', 'delta_1m_Visa_msaldototal', 'std_mpasivos_margen', 'std_cproductos', 'std_mcuenta_corriente', 'std_mcuentas_saldo', 'std_ctarjeta_visa_transacciones', 'std_mtarjeta_visa_consumo', 'std_mprestamos_personales', 'std_cpayroll_trx', 'std_ccallcenter_transacciones', 'std_ccajas_consultas', 'std_ctrx_quarter', 'std_Master_status', 'std_Master_mfinanciacion_limite', 'std_Visa_status', 'std_Visa_msaldototal']

In [13]:
lag_3 = ['lag_3_ctrx_quarter', 'lag_3_mcaja_ahorro', 'lag_3_mpasivos_margen', 'lag_3_mcuentas_saldo', 'lag_3_cpayroll_trx', 'lag_3_mprestamos_personales', 'lag_3_mtarjeta_visa_consumo', 'lag_3_mcuenta_corriente', 'lag_3_cprestamos_personales', 'lag_3_ctarjeta_visa_transacciones', 'lag_3_mrentabilidad_annual', 'lag_3_Visa_msaldopesos', 'lag_3_mpayroll', 'lag_3_Master_status', 'lag_3_mactivos_margen', 'lag_3_Visa_status', 'lag_3_ctarjeta_visa', 'lag_3_mcomisiones_mantenimiento', 'lag_3_ctarjeta_master', 'lag_3_tcallcenter', 'lag_3_mcomisiones', 'lag_3_Visa_msaldototal', 'lag_3_chomebanking_transacciones', 'lag_3_cproductos', 'lag_3_internet', 'lag_3_Master_Fvencimiento', 'lag_3_ccallcenter_transacciones', 'lag_3_ccajas_consultas', 'lag_3_Master_mfinanciacion_limite', 'lag_3_mautoservicio', 'lag_3_mrentabilidad', 'lag_3_Visa_mpagospesos', 'lag_3_Visa_Fvencimiento', 'lag_3_ccomisiones_otras', 'lag_3_Visa_mfinanciacion_limite', 'lag_3_cdescubierto_preacordado', 'lag_3_mtransferencias_recibidas', 'lag_3_Visa_mpagominimo', 'lag_3_ccomisiones_mantenimiento', 'lag_3_Visa_mconsumospesos', 'lag_3_mcomisiones_otras', 'lag_3_mcaja_ahorro_dolares', 'lag_3_Master_mlimitecompra', 'lag_3_cmobile_app_trx', 'lag_3_Master_mpagominimo', 'lag_3_ccajas_transacciones']
ranked_vars = ['ctrx_quarter_ranked', 'mcaja_ahorro_ranked', 'mpasivos_margen_ranked', 'mcuentas_saldo_ranked', 'cpayroll_trx_ranked', 'mprestamos_personales_ranked', 'mtarjeta_visa_consumo_ranked', 'mcuenta_corriente_ranked', 'cprestamos_personales_ranked', 'ctarjeta_visa_transacciones_ranked']

In [14]:
lag_2 = ['lag_2_ctrx_quarter', 'lag_2_mcaja_ahorro', 'lag_2_mpasivos_margen', 'lag_2_mcuentas_saldo', 'lag_2_cpayroll_trx', 'lag_2_mprestamos_personales', 'lag_2_mtarjeta_visa_consumo', 'lag_2_mcuenta_corriente', 'lag_2_cprestamos_personales', 'lag_2_ctarjeta_visa_transacciones', 'lag_2_mrentabilidad_annual', 'lag_2_Visa_msaldopesos', 'lag_2_mpayroll', 'lag_2_Master_status', 'lag_2_mactivos_margen', 'lag_2_Visa_status', 'lag_2_ctarjeta_visa', 'lag_2_mcomisiones_mantenimiento', 'lag_2_ctarjeta_master', 'lag_2_tcallcenter', 'lag_2_mcomisiones', 'lag_2_Visa_msaldototal', 'lag_2_chomebanking_transacciones', 'lag_2_cproductos', 'lag_2_internet', 'lag_2_Master_Fvencimiento', 'lag_2_ccallcenter_transacciones', 'lag_2_ccajas_consultas', 'lag_2_Master_mfinanciacion_limite', 'lag_2_mautoservicio', 'lag_2_mrentabilidad', 'lag_2_Visa_mpagospesos', 'lag_2_Visa_Fvencimiento', 'lag_2_ccomisiones_otras', 'lag_2_Visa_mfinanciacion_limite', 'lag_2_cdescubierto_preacordado', 'lag_2_mtransferencias_recibidas', 'lag_2_Visa_mpagominimo', 'lag_2_ccomisiones_mantenimiento', 'lag_2_Visa_mconsumospesos', 'lag_2_mcomisiones_otras', 'lag_2_mcaja_ahorro_dolares', 'lag_2_Master_mlimitecompra', 'lag_2_cmobile_app_trx', 'lag_2_Master_mpagominimo', 'lag_2_ccajas_transacciones']
lag_1 = ['lag_1_ctrx_quarter', 'lag_1_mcaja_ahorro', 'lag_1_mpasivos_margen', 'lag_1_mcuentas_saldo', 'lag_1_cpayroll_trx', 'lag_1_mprestamos_personales', 'lag_1_mtarjeta_visa_consumo', 'lag_1_mcuenta_corriente', 'lag_1_cprestamos_personales', 'lag_1_ctarjeta_visa_transacciones', 'lag_1_mrentabilidad_annual', 'lag_1_Visa_msaldopesos', 'lag_1_mpayroll', 'lag_1_Master_status', 'lag_1_mactivos_margen', 'lag_1_Visa_status', 'lag_1_ctarjeta_visa', 'lag_1_mcomisiones_mantenimiento', 'lag_1_ctarjeta_master', 'lag_1_tcallcenter', 'lag_1_mcomisiones', 'lag_1_Visa_msaldototal', 'lag_1_chomebanking_transacciones', 'lag_1_cproductos', 'lag_1_internet', 'lag_1_Master_Fvencimiento', 'lag_1_ccallcenter_transacciones', 'lag_1_ccajas_consultas', 'lag_1_Master_mfinanciacion_limite', 'lag_1_mautoservicio', 'lag_1_mrentabilidad', 'lag_1_Visa_mpagospesos', 'lag_1_Visa_Fvencimiento', 'lag_1_ccomisiones_otras', 'lag_1_Visa_mfinanciacion_limite', 'lag_1_cdescubierto_preacordado', 'lag_1_mtransferencias_recibidas', 'lag_1_Visa_mpagominimo', 'lag_1_ccomisiones_mantenimiento', 'lag_1_Visa_mconsumospesos', 'lag_1_mcomisiones_otras', 'lag_1_mcaja_ahorro_dolares', 'lag_1_Master_mlimitecompra', 'lag_1_cmobile_app_trx', 'lag_1_Master_mpagominimo', 'lag_1_ccajas_transacciones']
delta_1 = ['delta_1m_ctrx_quarter', 'delta_1m_mcaja_ahorro', 'delta_1m_mpasivos_margen', 'delta_1m_mcuentas_saldo', 'delta_1m_cpayroll_trx', 'delta_1m_mprestamos_personales', 'delta_1m_mtarjeta_visa_consumo', 'delta_1m_mcuenta_corriente', 'delta_1m_cprestamos_personales', 'delta_1m_ctarjeta_visa_transacciones', 'delta_1m_mrentabilidad_annual', 'delta_1m_Visa_msaldopesos', 'delta_1m_mpayroll', 'delta_1m_Master_status', 'delta_1m_mactivos_margen', 'delta_1m_Visa_status', 'delta_1m_ctarjeta_visa', 'delta_1m_mcomisiones_mantenimiento', 'delta_1m_ctarjeta_master', 'delta_1m_tcallcenter', 'delta_1m_mcomisiones', 'delta_1m_Visa_msaldototal', 'delta_1m_chomebanking_transacciones', 'delta_1m_cproductos', 'delta_1m_internet', 'delta_1m_Master_Fvencimiento', 'delta_1m_ccallcenter_transacciones', 'delta_1m_ccajas_consultas', 'delta_1m_Master_mfinanciacion_limite', 'delta_1m_mautoservicio', 'delta_1m_mrentabilidad', 'delta_1m_Visa_mpagospesos', 'delta_1m_Visa_Fvencimiento', 'delta_1m_ccomisiones_otras', 'delta_1m_Visa_mfinanciacion_limite', 'delta_1m_cdescubierto_preacordado', 'delta_1m_mtransferencias_recibidas', 'delta_1m_Visa_mpagominimo', 'delta_1m_ccomisiones_mantenimiento', 'delta_1m_Visa_mconsumospesos', 'delta_1m_mcomisiones_otras', 'delta_1m_mcaja_ahorro_dolares', 'delta_1m_Master_mlimitecompra', 'delta_1m_cmobile_app_trx', 'delta_1m_Master_mpagominimo', 'delta_1m_ccajas_transacciones']

In [15]:
delta_2 = ['delta_2m_ctrx_quarter', 'delta_2m_mcaja_ahorro', 'delta_2m_mpasivos_margen', 'delta_2m_mcuentas_saldo', 'delta_2m_cpayroll_trx', 'delta_2m_mprestamos_personales', 'delta_2m_mtarjeta_visa_consumo', 'delta_2m_mcuenta_corriente', 'delta_2m_cprestamos_personales', 'delta_2m_ctarjeta_visa_transacciones', 'delta_2m_mrentabilidad_annual', 'delta_2m_Visa_msaldopesos', 'delta_2m_mpayroll', 'delta_2m_Master_status', 'delta_2m_mactivos_margen', 'delta_2m_Visa_status', 'delta_2m_ctarjeta_visa', 'delta_2m_mcomisiones_mantenimiento', 'delta_2m_ctarjeta_master', 'delta_2m_tcallcenter', 'delta_2m_mcomisiones', 'delta_2m_Visa_msaldototal', 'delta_2m_chomebanking_transacciones', 'delta_2m_cproductos', 'delta_2m_internet', 'delta_2m_Master_Fvencimiento', 'delta_2m_ccallcenter_transacciones', 'delta_2m_ccajas_consultas', 'delta_2m_Master_mfinanciacion_limite', 'delta_2m_mautoservicio', 'delta_2m_mrentabilidad', 'delta_2m_Visa_mpagospesos', 'delta_2m_Visa_Fvencimiento', 'delta_2m_ccomisiones_otras', 'delta_2m_Visa_mfinanciacion_limite', 'delta_2m_cdescubierto_preacordado', 'delta_2m_mtransferencias_recibidas', 'delta_2m_Visa_mpagominimo', 'delta_2m_ccomisiones_mantenimiento', 'delta_2m_Visa_mconsumospesos', 'delta_2m_mcomisiones_otras', 'delta_2m_mcaja_ahorro_dolares', 'delta_2m_Master_mlimitecompra', 'delta_2m_cmobile_app_trx', 'delta_2m_Master_mpagominimo', 'delta_2m_ccajas_transacciones']

In [16]:
slopes = ['slope_2m_ctrx_quarter', 'slope_2m_mcaja_ahorro', 'slope_2m_mpasivos_margen', 'slope_2m_mcuentas_saldo', 'slope_2m_cpayroll_trx', 'slope_2m_mprestamos_personales', 'slope_2m_mtarjeta_visa_consumo', 'slope_2m_mcuenta_corriente', 'slope_2m_cprestamos_personales', 'slope_2m_ctarjeta_visa_transacciones', 'slope_2m_mrentabilidad_annual', 'slope_2m_Visa_msaldopesos', 'slope_2m_mpayroll', 'slope_2m_Master_status', 'slope_2m_mactivos_margen', 'slope_2m_Visa_status', 'slope_2m_ctarjeta_visa', 'slope_2m_mcomisiones_mantenimiento', 'slope_2m_ctarjeta_master', 'slope_2m_tcallcenter', 'slope_2m_mcomisiones', 'slope_2m_Visa_msaldototal', 'slope_2m_chomebanking_transacciones', 'slope_2m_cproductos', 'slope_2m_internet', 'slope_2m_Master_Fvencimiento', 'slope_2m_ccallcenter_transacciones', 'slope_2m_ccajas_consultas', 'slope_2m_Master_mfinanciacion_limite', 'slope_2m_mautoservicio', 'slope_2m_mrentabilidad', 'slope_2m_Visa_mpagospesos', 'slope_2m_Visa_Fvencimiento', 'slope_2m_ccomisiones_otras', 'slope_2m_Visa_mfinanciacion_limite', 'slope_2m_cdescubierto_preacordado', 'slope_2m_mtransferencias_recibidas', 'slope_2m_Visa_mpagominimo', 'slope_2m_ccomisiones_mantenimiento', 'slope_2m_Visa_mconsumospesos', 'slope_2m_mcomisiones_otras', 'slope_2m_mcaja_ahorro_dolares', 'slope_2m_Master_mlimitecompra', 'slope_2m_cmobile_app_trx', 'slope_2m_Master_mpagominimo', 'slope_2m_ccajas_transacciones']

In [17]:
ratios = ['ratio_mpasivos_margen_vs_prev', 'ratio_cproductos_vs_prev', 'ratio_mcuenta_corriente_vs_prev', 'ratio_cdescubierto_preacordado_vs_prev', 'ratio_mcuentas_saldo_vs_prev', 'ratio_ctarjeta_visa_transacciones_vs_prev', 'ratio_mtarjeta_visa_consumo_vs_prev', 'ratio_mprestamos_personales_vs_prev', 'ratio_cpayroll_trx_vs_prev', 'ratio_ccomisiones_mantenimiento_vs_prev', 'ratio_ccallcenter_transacciones_vs_prev', 'ratio_ccajas_consultas_vs_prev', 'ratio_ctrx_quarter_vs_prev', 'ratio_Master_status_vs_prev', 'ratio_Master_mfinanciacion_limite_vs_prev', 'ratio_Visa_status_vs_prev', 'ratio_Visa_msaldototal_vs_prev']

In [18]:
creadas = ['tc_saldo_total', 'es_empleado']

In [19]:
delta_3 = ['delta_3m_ctrx_quarter', 'delta_3m_mcaja_ahorro', 'delta_3m_mpasivos_margen', 'delta_3m_mcuentas_saldo', 'delta_3m_cpayroll_trx', 'delta_3m_mprestamos_personales', 'delta_3m_mtarjeta_visa_consumo', 'delta_3m_mcuenta_corriente', 'delta_3m_cprestamos_personales', 'delta_3m_ctarjeta_visa_transacciones', 'delta_3m_mrentabilidad_annual', 'delta_3m_Visa_msaldopesos', 'delta_3m_mpayroll', 'delta_3m_Master_status', 'delta_3m_mactivos_margen', 'delta_3m_Visa_status', 'delta_3m_ctarjeta_visa', 'delta_3m_mcomisiones_mantenimiento', 'delta_3m_ctarjeta_master', 'delta_3m_tcallcenter', 'delta_3m_mcomisiones', 'delta_3m_Visa_msaldototal', 'delta_3m_chomebanking_transacciones', 'delta_3m_cproductos', 'delta_3m_internet', 'delta_3m_Master_Fvencimiento', 'delta_3m_ccallcenter_transacciones', 'delta_3m_ccajas_consultas', 'delta_3m_Master_mfinanciacion_limite', 'delta_3m_mautoservicio', 'delta_3m_mrentabilidad', 'delta_3m_Visa_mpagospesos', 'delta_3m_Visa_Fvencimiento', 'delta_3m_ccomisiones_otras', 'delta_3m_Visa_mfinanciacion_limite', 'delta_3m_cdescubierto_preacordado', 'delta_3m_mtransferencias_recibidas', 'delta_3m_Visa_mpagominimo', 'delta_3m_ccomisiones_mantenimiento', 'delta_3m_Visa_mconsumospesos', 'delta_3m_mcomisiones_otras', 'delta_3m_mcaja_ahorro_dolares', 'delta_3m_Master_mlimitecompra', 'delta_3m_cmobile_app_trx', 'delta_3m_Master_mpagominimo', 'delta_3m_ccajas_transacciones']

In [20]:
means =['mean_ctrx_quarter', 'mean_mcaja_ahorro', 'mean_mpasivos_margen', 'mean_mcuentas_saldo', 'mean_cpayroll_trx', 'mean_mprestamos_personales', 'mean_mtarjeta_visa_consumo', 'mean_mcuenta_corriente', 'mean_cprestamos_personales', 'mean_ctarjeta_visa_transacciones', 'mean_mrentabilidad_annual', 'mean_Visa_msaldopesos', 'mean_mpayroll', 'mean_Master_status', 'mean_mactivos_margen', 'mean_Visa_status', 'mean_ctarjeta_visa', 'mean_mcomisiones_mantenimiento', 'mean_ctarjeta_master', 'mean_tcallcenter', 'mean_mcomisiones', 'mean_Visa_msaldototal', 'mean_chomebanking_transacciones', 'mean_cproductos', 'mean_internet', 'mean_Master_Fvencimiento', 'mean_ccallcenter_transacciones', 'mean_ccajas_consultas', 'mean_Master_mfinanciacion_limite', 'mean_mautoservicio', 'mean_mrentabilidad', 'mean_Visa_mpagospesos', 'mean_Visa_Fvencimiento', 'mean_ccomisiones_otras', 'mean_Visa_mfinanciacion_limite', 'mean_cdescubierto_preacordado', 'mean_mtransferencias_recibidas', 'mean_Visa_mpagominimo', 'mean_ccomisiones_mantenimiento', 'mean_Visa_mconsumospesos', 'mean_mcomisiones_otras', 'mean_mcaja_ahorro_dolares', 'mean_Master_mlimitecompra', 'mean_cmobile_app_trx', 'mean_Master_mpagominimo', 'mean_ccajas_transacciones']

In [21]:
maxs = ['max_3m_ctrx_quarter', 'max_3m_mcaja_ahorro', 'max_3m_mpasivos_margen', 'max_3m_mcuentas_saldo', 'max_3m_cpayroll_trx', 'max_3m_mprestamos_personales', 'max_3m_mtarjeta_visa_consumo', 'max_3m_mcuenta_corriente', 'max_3m_cprestamos_personales', 'max_3m_ctarjeta_visa_transacciones', 'max_3m_mrentabilidad_annual', 'max_3m_Visa_msaldopesos', 'max_3m_mpayroll', 'max_3m_Master_status', 'max_3m_mactivos_margen', 'max_3m_Visa_status', 'max_3m_ctarjeta_visa', 'max_3m_mcomisiones_mantenimiento', 'max_3m_ctarjeta_master', 'max_3m_tcallcenter', 'max_3m_mcomisiones', 'max_3m_Visa_msaldototal', 'max_3m_chomebanking_transacciones', 'max_3m_cproductos', 'max_3m_internet', 'max_3m_Master_Fvencimiento', 'max_3m_ccallcenter_transacciones', 'max_3m_ccajas_consultas', 'max_3m_Master_mfinanciacion_limite', 'max_3m_mautoservicio', 'max_3m_mrentabilidad', 'max_3m_Visa_mpagospesos', 'max_3m_Visa_Fvencimiento', 'max_3m_ccomisiones_otras', 'max_3m_Visa_mfinanciacion_limite', 'max_3m_cdescubierto_preacordado', 'max_3m_mtransferencias_recibidas', 'max_3m_Visa_mpagominimo', 'max_3m_ccomisiones_mantenimiento', 'max_3m_Visa_mconsumospesos', 'max_3m_mcomisiones_otras', 'max_3m_mcaja_ahorro_dolares', 'max_3m_Master_mlimitecompra', 'max_3m_cmobile_app_trx', 'max_3m_Master_mpagominimo', 'max_3m_ccajas_transacciones']

In [22]:
mins = ['min_ctrx_quarter', 'min_mcaja_ahorro', 'min_mpasivos_margen', 'min_mcuentas_saldo', 'min_cpayroll_trx', 'min_mprestamos_personales', 'min_mtarjeta_visa_consumo', 'min_mcuenta_corriente', 'min_cprestamos_personales', 'min_ctarjeta_visa_transacciones', 'min_mrentabilidad_annual', 'min_Visa_msaldopesos', 'min_mpayroll', 'min_Master_status', 'min_mactivos_margen', 'min_Visa_status', 'min_ctarjeta_visa', 'min_mcomisiones_mantenimiento', 'min_ctarjeta_master', 'min_tcallcenter', 'min_mcomisiones', 'min_Visa_msaldototal', 'min_chomebanking_transacciones', 'min_cproductos', 'min_internet', 'min_Master_Fvencimiento', 'min_ccallcenter_transacciones', 'min_ccajas_consultas', 'min_Master_mfinanciacion_limite', 'min_mautoservicio', 'min_mrentabilidad', 'min_Visa_mpagospesos', 'min_Visa_Fvencimiento', 'min_ccomisiones_otras', 'min_Visa_mfinanciacion_limite', 'min_cdescubierto_preacordado', 'min_mtransferencias_recibidas', 'min_Visa_mpagominimo', 'min_ccomisiones_mantenimiento', 'min_Visa_mconsumospesos', 'min_mcomisiones_otras', 'min_mcaja_ahorro_dolares', 'min_Master_mlimitecompra', 'min_cmobile_app_trx', 'min_Master_mpagominimo', 'min_ccajas_transacciones']

## Funciones

In [23]:
def get_dataset_mes(mes, features=None, kaggle = False):
    # Crear dataset Arrow
    dataset = ds.dataset(base_path + dataset_file, format="parquet")

    if isinstance(mes, int):
        filtro = (ds.field("foto_mes") == mes)
    elif isinstance(mes, list):
        filtro = ds.field("foto_mes").isin(mes)
    else:
        raise ValueError("mes debe ser int o list[int]")

    # Columnas obligatorias para etiquetas y pesos
    columnas_obligatorias = ["clase_ternaria"]

    # Si se pasan features, combinar con las obligatorias, sino usar todas
    if features is None:
        columnas = features_base + delta_1 + delta_2 + delta_3 + lag_1 + lag_2 + ratios + columnas_obligatorias + std
    if kaggle:
      columnas.append('numero_de_cliente')

    data = dataset.to_table(columns=columnas, filter=filtro).to_pandas()

    # --- Procesamiento mínimo ---
    data["clase_peso"] = 1.0
    data.loc[data["clase_ternaria"] == "BAJA+2", "clase_peso"] = 1.00002
    data.loc[data["clase_ternaria"] == "BAJA+1", "clase_peso"] = 1.00001
    data["clase_binaria1"] = (data["clase_ternaria"] == "BAJA+2").astype(int)
    data["clase_binaria2"] = (data["clase_ternaria"] != "CONTINUA").astype(int)

    data.columns = [re.sub(r"[^A-Za-z0-9_]", "_", c.strip()) for c in data.columns]

    X = data.drop(["clase_ternaria", "clase_peso", "clase_binaria1", "clase_binaria2"], axis=1)
    y_1 = data["clase_binaria1"]
    y_2 = data["clase_binaria2"]
    w = data["clase_peso"]

    # Liberar memoria
    del data
    gc.collect()
    return _, X, y_2, w, y_1


In [24]:
ganancia_acierto = 780000
costo_estimulo = 20000

def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

positions = []

def lgb_gan_eval_th(y_pred, data):
    weight = data.get_weight()

    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia_acum = np.cumsum(ganancia)

    max_gan = np.max(ganancia_acum)
    pos_max = np.argmax(ganancia_acum) + 1

    positions.append(pos_max)  # guarda la posición óptima de este fold

    return 'gan_eval', max_gan, True

In [25]:
def ganancia_prob(y_pred, y_true, th, prop=1):
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
    return ganancia[y_pred >= th].sum() / prop

In [26]:
def get_ganancia(y_pred, y_true, ganancia_acierto=ganancia_acierto, costo_estimulo=costo_estimulo):
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
    idx = np.argsort(y_pred)[::-1]
    ganancia = ganancia[idx]
    y_pred_ord = y_pred[idx]

    ganancia_cum = np.cumsum(ganancia)

    maxima_ganancia = np.max(ganancia_cum)
    envios = np.argmax(ganancia_cum)
    proba = y_pred_ord[envios]

    print(f"Máxima ganancia: {maxima_ganancia}\nEnvios: {envios}\nProba: {round(proba, 4)}")
    return maxima_ganancia, proba


In [27]:
def ensemble_rank_by_position_np(predictions_list, weights=None):
    """
    Combina predicciones de varios modelos usando ranking, solo con NumPy.

    Cada modelo contribuye con la posición de cada caso (mayor valor = mejor lugar),
    y el ranking final se obtiene sumando los lugares ponderados.

    Parámetros
    ----------
    predictions_list : list of np.ndarray
        Lista de arrays con las predicciones de cada modelo. Todos deben tener la misma longitud.
    weights : list of float, opcional
        Pesos para cada modelo. Si no se especifica, todos los modelos tienen el mismo peso.

    Retorna
    -------
    np.ndarray
        Array del mismo tamaño que las predicciones individuales, con valores que reflejan el ranking final.
        Solo importa el orden relativo.
    """

    if not predictions_list:
        raise ValueError("La lista de predicciones está vacía.")

    n_models = len(predictions_list)
    n_samples = len(predictions_list[0])

    if any(len(p) != n_samples for p in predictions_list):
        raise ValueError("Todas las predicciones deben tener la misma longitud.")

    if weights is None:
        weights = np.ones(n_models)
    else:
        weights = np.array(weights)
        if len(weights) != n_models:
            raise ValueError("La cantidad de pesos debe coincidir con la cantidad de modelos.")

    weighted_ranks = np.zeros(n_samples)

    for p, w in zip(predictions_list, weights):
        # Obtener ranking: argsort dos veces para obtener ranks
        ranks = np.empty_like(p, dtype=float)
        # Primer argsort: orden de menor a mayor
        order = np.argsort(-p)  # invertimos para que el mayor tenga menor número
        # Segundo argsort: posición en ranking
        ranks[order] = np.arange(1, n_samples + 1)  # 1 = mejor posición
        weighted_ranks += w * ranks

    # Invertimos para que mayores valores = mejor ranking (similar a predict)
    final_scores = -weighted_ranks
    return final_scores

def ensemble_predict(models_list, X_test, weights: list[float] = None):
    print(f"Prediciendo {len(X_test)} samples de {int(X_test.iloc[0].foto_mes)} con {len(models_list)} modelo(s)")
    predictions_list = []
    for model in models_list:
      predictions_list.append(model.predict(X_test[model.feature_name()]))

    return ensemble_rank_by_position_np(predictions_list, weights = weights)

In [28]:
def test_all_weight_combinations_parallel(predictions_list, y_true, weight_range, n_jobs=-1):
    """
    Prueba todas las combinaciones posibles de pesos, con rangos específicos por modelo.
    Devuelve la mejor combinación según la ganancia.

    Parámetros
    ----------
    predictions_list : list of np.ndarray
        Lista de predicciones de los modelos.
    y_true : np.ndarray
        Vector de etiquetas verdaderas.
    weight_range : list[tuple[int, int]]
        Lista de tuplas (min, max) por modelo. Ejemplo: [(0,5),(0,10),(0,0),(0,10)]
    n_jobs : int
        Número de procesos paralelos a usar (por defecto, usa todos los disponibles).

    Retorna
    -------
    dict
        {
            "mejores_pesos": list[int],
            "mejor_ganancia": float,
            "mejor_threshold": float,
            "resultados": list[dict]
        }
    """

    if len(weight_range) != len(predictions_list):
        raise ValueError("La cantidad de rangos de pesos debe coincidir con la cantidad de modelos.")

    # Generar los valores posibles para cada modelo
    valores_por_modelo = [range(r[0], r[1] + 1) for r in weight_range]

    # Crear todas las combinaciones posibles
    combinaciones = list(itertools.product(*valores_por_modelo))

    def evaluar_combinacion(weights):
        y_pred = ensemble_rank_by_position_np(predictions_list, weights=weights)
        ganancia, thr = get_ganancia(y_pred, y_true)
        return {"weights": weights, "ganancia": ganancia, "threshold": thr}

    resultados = Parallel(n_jobs=n_jobs, backend="loky")(delayed(evaluar_combinacion)(w) for w in combinaciones)

    resultados_ordenados = sorted(resultados, key=lambda x: x["ganancia"], reverse=True)
    mejor = resultados_ordenados[0]

    print("\nTop 5 combinaciones por ganancia:")
    for i, r in enumerate(resultados_ordenados[:5], start=1):
        print(f"{i}. Pesos: {r['weights']} | Ganancia: {r['ganancia']:.2f} | Threshold: {r['threshold']:.4f}")

    print(f"\nMejores pesos globales: {mejor['weights']} | Ganancia: {mejor['ganancia']:.2f} | Threshold: {mejor['threshold']:.4f}")

    return {
        "mejores_pesos": mejor["weights"],
        "mejor_ganancia": mejor["ganancia"],
        "mejor_threshold": mejor["threshold"],
        "resultados": resultados_ordenados
    }


# Feature importance

In [ ]:
seed = semillas[0]

params = {
  "max_bin": 31,
  "num_leaves": 31,
  "learning_rate": 0.01,
  "min_data_in_leaf": 50,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 300,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True,
  "force_row_wise": True
}

In [ ]:
_, X_train, y_train, w_train, _ = get_dataset_mes([202102,202103,202104])

In [ ]:
# X_train = X_train[columns_old]
# X_train = X_train.loc[:, ~X_train.columns.str.contains(r'(?:lag_[32]|delta_[3]|numero_de_cliente|foto_mes)')]

In [ ]:
train_data = lgb.Dataset(X_train,
                          label=y_train,
                          weight=w_train)

print(f"Entrenando modelo con parámetros: {params}")
print(X_train.shape)
model = lgb.train(params = params,
                  train_set = train_data)

In [ ]:
importance_type = 'gain'
importance_type = 'split'

importances = model.feature_importance(importance_type = importance_type)
feature_names = X_train.columns.tolist()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
importance_df[importance_df['importance'] > 0]

In [ ]:
importances = model.feature_importance(importance_type = 'gain')
importances_gain = pd.DataFrame({'feature': feature_names, 'importance': importances})
importances_gain = importances_gain.sort_values('importance', ascending=False)
importances_gain[importances_gain['importance'] > 0]

importances = model.feature_importance(importance_type = 'split')
importances_split = pd.DataFrame({'feature': feature_names, 'importance': importances})
importances_split = importances_split.sort_values('importance', ascending=False)
importances_split[importances_split['importance'] > 0]

imp_feat = list(set(list(importances_gain.head(200).feature.values) + list(importances_split.head(200).feature.values)))

In [ ]:
len(imp_feat)

In [ ]:
importance_df.to_csv("importance.csv")

# Optuna

In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
_, X_sub, y_sub, w_sub, _ = get_dataset_mes([202103])
# X_sub, _, y_sub, _, w_sub, _ = train_test_split(
#     X_opt, y_opt, w_opt,
#     test_size=1,
#     stratify=y_opt,
#     random_state=semillas[0]
# )
# X_sub = X_sub
# print(X_sub.shape)
# del X_opt, y_opt, w_opt

In [ ]:
def objective(trial):

    # Sugerencias (todas al nivel superior, sin condicionales que creen distribuciones dinámicas)
    max_bin = trial.suggest_int('max_bin', 30, 150, step=10)
    num_leaves = trial.suggest_int('num_leaves', 30, 150, step=10)
    learning_rate = trial.suggest_float('learning_rate', high = 0.1, low = 0.005, log = True)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 200, step=10)
    feature_fraction = trial.suggest_categorical('feature_fraction', [0.5, 0.75, 1.0])
    bagging_freq = trial.suggest_categorical('bagging_freq', [0, 5])
    # definimos bagging_fraction siempre (luego lo ignoramos si bagging_freq == 0)
    bagging_fraction = trial.suggest_categorical('bagging_fraction', [0.5, 1.0])
    option_idx = trial.suggest_int("feature_set", 0, 3)
    if option_idx == 0:
        features = columns_old
    elif option_idx == 1:
        features = features_106
    elif option_idx == 2:
        features = columns_old + ['numero_de_cliente']
    else:
        features = list(X_sub.columns.drop(["foto_mes", "numero_de_cliente"]))


    # si bagging_freq == 0, forzamos bagging_fraction a 1.0 (no requiere nueva sugerencia)
    if bagging_freq == 0:
        actual_bagging_fraction = 1.0
    else:
        actual_bagging_fraction = bagging_fraction

    train_data = lgb.Dataset(X_sub[features],
                          label=y_sub,
                          weight=w_sub)

    params = {
        'objective': 'binary',
        'metric': 'auc',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': True,
        'max_bin': max_bin,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': actual_bagging_fraction,
        'bagging_freq': bagging_freq,
        'boosting_type': 'gbdt',
        'seed': semillas[0],
        'verbose': -1,
    }

    cv_results = lgb.cv(
        params,
        train_data,
        metrics = "auc",
        num_boost_round=800,
        callbacks=[lgb.early_stopping(first_metric_only = True, stopping_rounds=50)],
        stratified=True,
        nfold=5,
        seed=semillas[0],
        feval=lgb_gan_eval,
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    max_auc = max(cv_results['valid auc-mean'])
    best_iter_gan = cv_results['valid gan_eval-mean'].index(max_gan) + 1
    best_iter_auc = cv_results['valid auc-mean'].index(max_auc) + 1

    trial.set_user_attr("best_iter_gan", best_iter_gan)
    trial.set_user_attr("best_iter_auc", best_iter_auc)
    trial.set_user_attr("max_gan", max_gan)

    return max_auc


storage_name = "sqlite:///" + db_path + "base_04_10_v3.db"
study_name = "auc_marzo"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
study.optimize(n_trials = 200, func = objective, n_jobs = -1)

# Ensamble

In [ ]:
seed = 77

## Entrenamiento

In [ ]:
params = {
  "max_bin": 31,
  "num_leaves": 31,
  "learning_rate": 0.01,
  "min_data_in_leaf": 20,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 50,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True,
  "force_row_wise": True,
  "seed":seed+1
}

_, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103])

train_data = lgb.Dataset(X_train,
                          label=y_train,
                          weight=w_train)

print(f"Entrenando modelo con parámetros: {params}")
print(X_train.shape)
del X_train, y_train, w_train
model_e1 = lgb.train(params = params,
                  train_set = train_data)
del train_data

In [ ]:
params = {
  "max_bin": 31,
  "num_leaves": 31,
  "learning_rate": 0.01,
  "min_data_in_leaf": 20,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 50,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True,
  "force_row_wise": True,
  "seed":seed+2
}

_, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103])

#w_train = np.where(w_train == 1.00002, 100, w_train)
#y_train = np.where(w_train == 1.00002, 1, 0)

train_data = lgb.Dataset(X_train,
                          label=y_train,
                          weight=w_train)

print(f"Entrenando modelo con parámetros: {params}")
print(X_train.shape)
del X_train, y_train, w_train
model_e2 = lgb.train(params = params,
                  train_set = train_data)
del train_data

In [ ]:
params = {
  "max_bin": 31,
  "num_leaves": 31,
  "learning_rate": 0.01,
  "min_data_in_leaf": 20,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 50,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True,
  "force_row_wise": True,
  "seed":seed+3
}
_, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103])

train_data = lgb.Dataset(X_train,
                          label=y_train,
                          weight=w_train)

print(f"Entrenando modelo con parámetros: {params}")
print(X_train.shape)
del X_train, y_train, w_train
model_e3 = lgb.train(params = params,
                  train_set = train_data)
del train_data

In [ ]:
# params = {
#   "max_bin": 31,
#   "num_leaves": 31,
#   "learning_rate": 0.01,
#   "min_data_in_leaf": 20,
#   "feature_fraction": 0.5,
#   "bagging_freq": 5,
#   "bagging_fraction": 0.5,
#   "feature_pre_filter": True,
#   "num_boost_round": 50,
#   "boosting_type": "gbdt",
#   "verbose": 1,
#   "objective": "binary",
#   "boost_from_average": True,
#   "force_row_wise": True,
#   "seed":4
# }


# _, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103])
# X_train = X_train


# train_data = lgb.Dataset(X_train,
#                           label=y_train,
#                           weight=w_train)

# print(f"Entrenando modelo con parámetros: {params}")
# print(X_train.shape)
# del X_train, y_train, w_train
# model_e4 = lgb.train(params = params,
#                   train_set = train_data)
# del train_data

## Predicción

In [ ]:
_, X_test, _, _, y_true = get_dataset_mes([202104])

In [ ]:
predicciones_modelos = {}

for i, model in enumerate([model_e1, model_e2, model_e3], start=1):
    predicciones_modelos[f"modelo_{i}"] = model.predict(X_test[model.feature_name()])


In [ ]:
# Ensamble usando las predicciones ya calculadas
pred_list = [predicciones_modelos[f"modelo_{i}"] for i in range(1, 4)]
y_pred_e = np.mean(pred_list, axis = 0)#ensemble_rank_by_position_np(pred_list, weights=[1, 1, 1])
ganancia_e, thr_e = get_ganancia(y_pred_e, y_true)
resultados = {
    "ensamble": {"ganancia": ganancia_e, "threshold": thr_e, "y_pred": y_pred_e}
}

# Evaluar cada modelo individual usando las predicciones ya calculadas
for i in range(1, 4):
    y_pred = predicciones_modelos[f"modelo_{i}"]
    ganancia, thr = get_ganancia(y_pred, y_true)
    resultados[f"modelo_{i}"] = {"ganancia": ganancia, "threshold": thr, "y_pred": y_pred}

# Seleccionar el mejor modelo (sin incluir el ensamble)
mejor_modelo = max([f"modelo_{i}" for i in range(1, 4)], key=lambda m: resultados[m]["ganancia"])
mejor_ganancia = resultados[mejor_modelo]["ganancia"]
mejor_threshold = resultados[mejor_modelo]["threshold"]
mejor_y_pred = resultados[mejor_modelo]["y_pred"]

In [ ]:
# best_combo = test_all_weight_combinations_parallel(
#     predictions_list=pred_list,
#     y_true=y_true,
#     weight_range=[(0,5), (0,5), (0,5)]
# )

In [ ]:
sss_futuro = StratifiedShuffleSplit(n_splits=50,
                             test_size=0.3,
                             random_state=seed)

modelos = {"mejor_modelo":{"pred": mejor_y_pred, "th": mejor_threshold}, "ensamble":{"pred":y_pred_e, "th": resultados["ensamble"]["threshold"]}}
rows = []
for private_index, public_index in sss_futuro.split(X_test, y_true):
  row = {}
  for name, y_pred in modelos.items():
    row[name + "_public"] = ganancia_prob(modelos[name]["pred"][public_index], y_true.iloc[public_index], th = modelos[name]["th"], prop = 0.3)
    row[name + "_private"] = ganancia_prob(modelos[name]["pred"][private_index], y_true.iloc[private_index], th = modelos[name]["th"], prop = 0.7)
  rows.append(row)
df_lb = pd.DataFrame(rows)

df = pd.DataFrame()
df['best_public'] = df_lb.filter(regex='_public').idxmax(axis=1)
df['best_private'] = df_lb.filter(regex='_private').idxmax(axis=1)

pd.crosstab(df['best_public'], df['best_private'])

## Punto de corte

In [ ]:
_, X_test, _, _, y_true = get_dataset_mes([202103])

predicciones_modelos = {}

for i, model in enumerate([model_e1, model_e2, model_e3], start=1):
    predicciones_modelos[f"modelo_{i}"] = model.predict(X_test[model.feature_name()])

In [ ]:
pred_list = [predicciones_modelos[f"modelo_{i}"] for i in range(1, 4)]
y_pred_e = ensemble_rank_by_position_np(pred_list, weights=[1, 1, 1])

# Resultado por cliente si lo estimuláramos
ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)

# Se ordena por mayor a menor probabilidad
idx = np.argsort(y_pred_e)[::-1]

ganancia = ganancia[idx]
y_pred = y_pred_e[idx] # Ordenados de mayor a menor probabilidad

ganancia_cum = np.cumsum(ganancia)

In [ ]:
maxima_ganancia = np.max(ganancia_cum)
envios = np.argmax(ganancia_cum)
proba = y_pred[envios]

print(f"Maxima ganancia: {maxima_ganancia}\nEnvios:{envios}\nProba:{round(proba,4)}")

# CV manual

## modelo 1

In [ ]:
seed = 24

In [ ]:
params_cv_1 = {
  "max_bin": 50,
  "num_leaves": 50,
  "learning_rate": 0.007,
  "min_data_in_leaf": 20,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 1500,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "boost_from_average": True,
  "force_row_wise": True
}

In [ ]:
_, X_cv_1, y_cv_1, w_cv_1, _ = get_dataset_mes([202104])

mask = y_cv_1[y_cv_1 != 0]

w_cv_1 = np.where(w_cv_1 == 1.00001, 0, np.where(w_cv_1 == 1.00002, 1, w_cv_1))

# Ahora descartar las filas donde y_cv_1 == 0
mask = y_cv_1 != 0
X_cv_1 = X_cv_1[mask]
y_cv_1 = y_cv_1[mask]
w_cv_1 = w_cv_1[mask]

#X_cv_1 = X_cv_1[columns_old]
#X_cv_1 = X_cv_1.loc[:, ~X_cv_1.columns.str.contains(r'(?:lag_[32]|delta_[3]|numero_de_cliente|foto_mes)')]
#X_cv_1 = X_cv_1.loc[:, ~X_cv_1.columns.str.contains(r'(?:lag_[3]|delta_[3]|numero_de_cliente|foto_mes)')]

In [ ]:
np.unique(w_cv_1)

In [ ]:
train_data_1 = lgb.Dataset(X_cv_1, label=w_cv_1)

print(X_cv_1.shape)

cv_1 = lgb.cv(
    params = params_cv_1,
    metrics = "auc",
    train_set = train_data_1,
    callbacks=[
        lgb.early_stopping(first_metric_only = True, stopping_rounds=50),
        lgb.log_evaluation(period=50),
    ],
    stratified=True,
    nfold=5,
    seed=seed
)

In [ ]:
print(positions[-50:])

In [ ]:
# print(np.max(cv_1["valid gan_eval-mean"]))
print(np.max(cv_1["valid auc-mean"]))

In [ ]:
del X_cv_1, train_data_1, y_cv_1, w_cv_1

## modelo 2

In [ ]:
params_cv_2 = {
  "max_bin": 100,
  "num_leaves": 75,
  "learning_rate": 0.01,
  "min_data_in_leaf": 50,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 50,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "seed":seed,
  "boost_from_average": True
}

In [ ]:
_, X_cv_2, y_cv_2, w_cv_2, _ = get_dataset_mes([202104])

X_cv_2 = X_cv_2

In [ ]:
train_data_2 = lgb.Dataset(X_cv_2, label=y_cv_2, weight=w_cv_2)

print(X_cv_2.shape)

cv_2 = lgb.cv(
    params = params_cv_2,
    feval=lgb_gan_eval,
    train_set = train_data_2,
    callbacks=[
        lgb.log_evaluation(period=50),
    ],
    stratified=True,
    nfold=5,
    seed=seed
)

print(np.max(cv_2["valid gan_eval-mean"]))

In [ ]:
print(np.max(cv_1["valid gan_eval-mean"]))
print(np.max(cv_2["valid gan_eval-mean"]))

In [ ]:
del X_cv_2, train_data_2, y_cv_2, w_cv_2

# Comparación 2 modelos

In [ ]:
seed = 68

In [ ]:
fixed_params = {
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "seed": seed,
  "boost_from_average": True
}

In [ ]:
#prueba

params_model_1 = {
  "max_bin": 80,
  "num_leaves": 80,
  "learning_rate": 0.007,
  "min_data_in_leaf": 180,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 200,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "force_row_wise": True,
  "boost_from_average": True,
  "seed":seed
}

params_model_1 = params_model_1 | fixed_params

In [ ]:
params_model_2 = {
  "max_bin": 80,
  "num_leaves": 80,
  "learning_rate": 0.007,
  "min_data_in_leaf": 180,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 200,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "force_row_wise": True,
  "boost_from_average": True,
  "seed":seed,
}

params_model_2 = params_model_2 | fixed_params

In [ ]:
vars_to_keep = features_base + delta_1 + delta_2

_, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103], features = vars_to_keep)
_, X_test, _, _, y_true = get_dataset_mes([202104], features = vars_to_keep)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# w_train_1 = w_train.copy()

# mask_202103 = X_train["foto_mes"] == 202103
# w_train_1[mask_202103] = w_train_1[mask_202103] * 1.3
w_train_1 = w_train
w_train_2 = w_train

In [ ]:
y_train_1 = y_train[y_train == 1]
y_train_0 = y_train[y_train == 0]

# Tomar el 50% de los que valen 1
y_train_0_sample = y_train_0.sample(frac=0.5, random_state=seed)

# Combinar
y_train_filtered = pd.concat([y_train_1, y_train_0_sample]).sort_index()

# Filtrar x_train en consecuencia
X_train_1 = X_train.loc[y_train_filtered.index]
X_train_2 = X_train.loc[y_train_filtered.index]

In [ ]:
X_train_1 = X_train_1[features_base + delta_1 + ratios] #[[col for col in columns_old if col not in lag_1]]
X_train_2 = X_train_2[features_base + delta_1]

def train_model(params, X_train, y_train, w_train):
    print(f"\nEntrenando modelo con parámetros: {params}\nShape: {X_train.shape}")
    train_data = lgb.Dataset(X_train, label=y_train, weight=w_train)

    try:
      del X_train_1
    except Exception as e:
      pass
    try:
      del X_train_2
    except Exception as e:
      pass

    model = lgb.train(params=params, train_set=train_data)
    return model

try:
  print("Eliminando data...")
  del X_train
except Exception as e:
  print(e)

with parallel_backend('threading'):
    models = Parallel(n_jobs=-1)(
        delayed(train_model)(p, X, y_train[X.index], w_train[X.index])
        for p, X, w_train in [(params_model_1, X_train_1, w_train_1), (params_model_2, X_train_2, w_train_2)]
    )

model_1, model_2 = models

In [ ]:
y_pred_1 = model_1.predict(X_test[model_1.feature_name()])
y_pred_2 = model_2.predict(X_test[model_2.feature_name()])

In [ ]:
def ganancia_prob_th(y_pred, y_true, prop=1):
    # Calcular ganancia individual para cada observación
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)

    # Ordenar por probabilidad descendente
    idx = np.argsort(y_pred)[::-1]
    ganancia_ordenada = ganancia[idx]
    y_pred_ordenado = y_pred[idx]

    # Ganancia acumulada
    ganancia_cum = np.cumsum(ganancia_ordenada) / prop

    # Encontrar el máximo
    idx_max = ganancia_cum.argmax()
    mejor_ganancia = ganancia_cum[idx_max]
    mejor_threshold = y_pred_ordenado[idx_max]
    mejor_envios = idx_max + 1

    print(f"Mejor punto de corte: {mejor_threshold:.6f}")
    print(f"Envíos: {mejor_envios}")
    print(f"Ganancia máxima: {mejor_ganancia:.2f}")

    return mejor_ganancia, mejor_threshold

# Uso
print("Modelo 1:")
ganancia_max_1, threshold_model_1 = ganancia_prob_th(y_pred_1, y_true)

print("\nModelo 2:")
ganancia_max_2, threshold_model_2 = ganancia_prob_th(y_pred_2, y_true)

In [ ]:
def ganancia_prob(y_pred, y_true, th, prop=1):
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
    return ganancia[y_pred >= th].sum() / prop

sss_futuro = StratifiedShuffleSplit(n_splits=50,
                             test_size=0.3,
                             random_state=seed)
modelos = {"model1":{"pred": y_pred_1, "th":threshold_model_1}, "model2":{"pred":y_pred_2, "th": threshold_model_2}}
rows = []
for private_index, public_index in sss_futuro.split(X_test, y_true):
  row = {}
  for name, y_pred in modelos.items():
    row[name + "_public"] = ganancia_prob(modelos[name]["pred"][public_index], y_true.iloc[public_index], th = modelos[name]["th"], prop = 0.3)
    row[name + "_private"] = ganancia_prob(modelos[name]["pred"][private_index], y_true.iloc[private_index], th = modelos[name]["th"], prop = 0.7)
  rows.append(row)
df_lb = pd.DataFrame(rows)

In [ ]:
df_lb_long = df_lb.reset_index()
df_lb_long = df_lb_long.melt(id_vars=['index'], var_name='model_type', value_name='ganancia')
df_lb_long[['modelo', 'tipo']] = df_lb_long['model_type'].str.split('_', expand=True)
df_lb_long = df_lb_long[['ganancia', 'tipo', 'modelo']]

## Resultados

In [ ]:
g = sns.FacetGrid(df_lb_long, col="tipo", row="modelo", aspect=2)
g.map(sns.histplot, "ganancia", kde=True)
plt.show()

In [ ]:
df = pd.DataFrame()
df['best_public'] = df_lb.filter(regex='_public').idxmax(axis=1)
df['best_private'] = df_lb.filter(regex='_private').idxmax(axis=1)

pd.crosstab(df['best_public'], df['best_private'])

In [ ]:
def threshold_optimo_por_ganancia(y_pred, y_true, ganancia_acierto, costo_estimulo):
    """Devuelve el threshold que maximiza la ganancia acumulada."""
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
    idx = np.argsort(y_pred)[::-1]
    ganancia_cum = np.cumsum(ganancia[idx])
    envios_opt = np.argmax(ganancia_cum)
    y_pred_ordenado = y_pred[idx]
    th_opt = y_pred_ordenado[envios_opt]
    return th_opt, ganancia_cum[envios_opt]

sss_futuro = StratifiedShuffleSplit(n_splits=50, test_size=0.3, random_state=seed)

thresholds_privados = []
thresholds_publicos = []

for private_index, public_index in sss_futuro.split(X_test, y_true):
    # Privado
    th_priv, gan_priv = threshold_optimo_por_ganancia(
        y_pred_1[private_index],
        y_true.iloc[private_index],
        ganancia_acierto,
        costo_estimulo
    )
    thresholds_privados.append(th_priv)

    # Público
    th_pub, gan_pub = threshold_optimo_por_ganancia(
        y_pred_1[public_index],
        y_true.iloc[public_index],
        ganancia_acierto,
        costo_estimulo
    )
    thresholds_publicos.append(th_pub)

print(f"Threshold privado promedio: {np.mean(thresholds_privados):.4f}")
print(f"Threshold público promedio: {np.mean(thresholds_publicos):.4f}")

In [ ]:
del X_train_1, X_train_2

# Punto de corte

In [ ]:
seed = 254

In [ ]:
params = {
  "max_bin": 80,
  "num_leaves": 80,
  "learning_rate": 0.007,
  "min_data_in_leaf": 180,
  "feature_fraction": 0.5,
  "bagging_freq": 5,
  "bagging_fraction": 0.5,
  "feature_pre_filter": True,
  "num_boost_round": 500,
  "boosting_type": "gbdt",
  "verbose": 1,
  "objective": "binary",
  "force_row_wise": True,
  "boost_from_average": True,
  "seed":seed
}

In [ ]:
_, X_train, y_train, w_train, _ = get_dataset_mes([202101])
_, X_test, _, _, y_test_binaria1 = get_dataset_mes([202104])


train_data = lgb.Dataset(X_train[features_base],
                          label=y_train,
                          weight=w_train)
del X_train, y_train, w_train
print(f"Entrenando model con parámetros: {params}")

model_pc = lgb.train(params = params,
                  train_set = train_data)

In [ ]:
# pd.DataFrame({"feature": X_train.columns, "importance": model.feature_importance(importance_type = "gain")}).sort_values(by = "importance", ascending = False)

In [ ]:
y_pred = model_pc.predict(X_test[model_pc.feature_name()])

In [ ]:
# Resultado por cliente si lo estimuláramos
ganancia = np.where(y_test_binaria1 == 1, ganancia_acierto, 0) - np.where(y_test_binaria1 == 0, costo_estimulo, 0)

# Se ordena por mayor a menor probabilidad
idx = np.argsort(y_pred)[::-1]

ganancia = ganancia[idx]
y_pred_ordenado = y_pred[idx] # Ordenados de mayor a menor probabilidad

ganancia_cum = np.cumsum(ganancia)

In [ ]:
print(ganancia_cum[0:100])

In [ ]:
piso_envios = 4000
techo_envios = 20000

plt.figure(figsize=(10, 6))
plt.plot(y_pred_ordenado[piso_envios:techo_envios], ganancia_cum[piso_envios:techo_envios], label='Ganancia LGBM')
plt.title('Curva de Ganancia')
plt.xlabel('Predicción de probabilidad')
plt.ylabel('Ganancia')
plt.axvline(x=0.025, color='g', linestyle='--', label='Punto de corte a 0.025')
plt.legend()
plt.show()


In [ ]:
maxima_ganancia = np.max(ganancia_cum)
envios = np.argmax(ganancia_cum)
proba = y_pred_ordenado[envios]

print(f"Maxima ganancia: {maxima_ganancia}\nEnvios:{envios}\nProba:{round(proba,4)}")

In [ ]:
def threshold_optimo_por_ganancia(y_pred, y_true, ganancia_acierto, costo_estimulo):
    """Devuelve el threshold que maximiza la ganancia acumulada."""
    ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
    idx = np.argsort(y_pred)[::-1]
    ganancia_cum = np.cumsum(ganancia[idx])
    envios_opt = np.argmax(ganancia_cum)
    y_pred_ordenado = y_pred[idx]
    th_opt = y_pred_ordenado[envios_opt]
    return th_opt, ganancia_cum[envios_opt]

sss_futuro = StratifiedShuffleSplit(n_splits=50, test_size=0.3, random_state=seed)

thresholds_privados = []
thresholds_publicos = []

for private_index, public_index in sss_futuro.split(X_test, y_test_binaria1):
    # Privado
    th_priv, gan_priv = threshold_optimo_por_ganancia(
        y_pred[private_index],
        y_test_binaria1.iloc[private_index],
        ganancia_acierto,
        costo_estimulo
    )
    thresholds_privados.append(th_priv)

    # Público
    th_pub, gan_pub = threshold_optimo_por_ganancia(
        y_pred[public_index],
        y_test_binaria1.iloc[public_index],
        ganancia_acierto,
        costo_estimulo
    )
    thresholds_publicos.append(th_pub)

print(f"Threshold privado promedio: {np.mean(thresholds_privados):.4f}")
print(f"Threshold público promedio: {np.mean(thresholds_publicos):.4f}")

In [ ]:
del X_test, y_true

# Train final

In [29]:
_, X_kaggle, _, _, _ = get_dataset_mes(202106, kaggle = True)

In [30]:
semillas = [0,1,24,19,12,2018,1918,1900]

In [31]:
subset = list(set(seleccion + features_base + ratios + delta_1 + delta_2 + lag_1))
#subset = seleccion

_, X_train, y_train, w_train, _ = get_dataset_mes([202101, 202102, 202103, 202104])


for seed in semillas[0:1]:
  params = {
    "max_bin": 50,
    "num_leaves": 50,
    "learning_rate": 0.01,
    "min_data_in_leaf": 200,
    "feature_fraction": 0.5,
    "bagging_freq": 5,
    "bagging_fraction": 0.5,
    "feature_pre_filter": True,
    "num_boost_round": 1000,
    "boosting_type": "gbdt",
    "verbose": 1,
    "objective": "binary",
    "force_row_wise": True,
    "boost_from_average": True,
    "seed":seed,
    'is_unbalance':True
  }

  if subset:
    X_train = X_train[subset]

  train_data = lgb.Dataset(X_train,
                            label=y_train,
                            weight=w_train)

  print(f"Entrenando modelo con parámetros: {params}")
  print(X_train.shape)

  model = lgb.train(params = params,
                    train_set = train_data)

  predictions = model.predict(X_kaggle[model.feature_name()])
  np.save(f'/content/drive/MyDrive/EyF/predicciones/{seed}.npy', predictions)

  importance_type = 'split'
  importance_type = 'gain'

  importances = model.feature_importance(importance_type = importance_type)
  feature_names = model.feature_name()
  importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
  importance_df = importance_df.sort_values('importance', ascending=False)
  importance_df[importance_df['importance'] > 0]

  print(importance_df.head(50))

Entrenando modelo con parámetros: {'max_bin': 50, 'num_leaves': 50, 'learning_rate': 0.01, 'min_data_in_leaf': 200, 'feature_fraction': 0.5, 'bagging_freq': 5, 'bagging_fraction': 0.5, 'feature_pre_filter': True, 'num_boost_round': 1000, 'boosting_type': 'gbdt', 'verbose': 1, 'objective': 'binary', 'force_row_wise': True, 'boost_from_average': True, 'seed': 0, 'is_unbalance': True}
(650209, 323)
[LightGBM] [Info] Number of positive: 7385, number of negative: 642824
[LightGBM] [Info] Total Bins 11870
[LightGBM] [Info] Number of data points in the train set: 650209, number of used features: 321
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011358 -> initscore=-4.466405
[LightGBM] [Info] Start training from score -4.466405
                                 feature    importance
265                         cpayroll_trx  4.129510e+06
158                             mpayroll  1.786361e+06
125                     Visa_msaldopesos  1.733831e+06
136                         std_mpayroll  1.21

# Ensamble

In [38]:
predicciones = list()
for seed in semillas[0:1]:
  path = f'/content/drive/MyDrive/EyF/predicciones/{seed}.npy'
  preds = np.load(path)
  predicciones.append(preds)

In [39]:
np.mean(predicciones, axis = 0).shape

(164313,)

In [40]:
X_kaggle["numero_de_cliente"]

,numero_de_cliente
0,249302860
1,249363168
2,249378334
3,249427711
4,249509684
...,...
164308,1593917771
164309,1595838016
164310,1597522333
164311,1598425905


In [41]:
th = 0.6
X_kaggle["proba"] = np.mean(predicciones, axis = 0)
X_kaggle["Predicted"] = (X_kaggle["proba"] > th).astype(int)

In [42]:
X_kaggle[["Predicted"]].value_counts()

,count
Predicted,
0,151501
1,12812


In [43]:
X_kaggle[["numero_de_cliente", "Predicted"]].to_csv("ultima_19.csv", index = False)